In [109]:
import os, re
import pyconll
import json
import pandas as pd
import random
import openai
import os
import pandas as pd
import re
import json
from tqdm import tqdm

# import sys
# sys.path.append('../protosp01')


In [2]:
# navigate to protosp01 skill extraction directory to import libraries
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("../protosp01/skillExtract")
assert os.getcwd().split("/")[-1] == "skillExtract"


from utils import *
from prompt_template import PROMPT_TEMPLATES

In [3]:
from split_words import Splitter

splitter = Splitter()
list(splitter.split_compound("Sprachkenntnisse")[0][1:])

['Sprach', 'Kenntnisse']

# get taxonomy elements

In [11]:
# navigate to data directory
if os.getcwd().split("/")[-1] == "skillExtract":
    os.chdir("../../data")
assert os.getcwd().split("/")[-1] == "data"

In [ ]:
# read from excel file each tab
# write to csv file each tab

taxonomy = pd.read_excel("taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="V4")
print(len(taxonomy))
taxonomy = taxonomy.dropna(subset=["Type Level 1", "Type Level 2", "Definition"])
print(len(taxonomy))
taxonomy.insert(0, "unique_id", list(range(len(taxonomy))))
#TODO: make unique ID unique to the lowest level of the taxonomy

taxonomy.to_csv("taxonomy/taxonomy_V4.csv", index=False)

n = len(taxonomy)

tech = pd.read_excel(
    "taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="Technologies"
)
print(len(tech))
certif = pd.read_excel(
    "taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="Certifications"
)
print(len(certif))
lang = pd.read_excel("taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="Languages")
print(len(lang))

tech_certif_lang = pd.concat([tech, certif, lang], ignore_index=True)
tech_certif_lang.insert(
    0, "unique_id", list([i + n for i in range(len(tech_certif_lang))])
)
# TODO: make unique ID increment from taxonomy ID

tech_certif_lang.head()
print(len(tech_certif_lang))
tech_certif_lang.to_csv("taxonomy/tech_certif_lang.csv", index=False)

In [6]:
tech_certif_lang.head()

,Level 1,Level 2,Level 1.5\n,unique_id
0,Technologies,Microsoft Excel,NaN,0
1,Technologies,Microsoft Office software,NaN,1
2,Technologies,Microsoft PowerPoint,NaN,2
3,Technologies,Microsoft Outlook,NaN,3
4,Technologies,Microsoft Word,NaN,4


# Extend taxonomy 

In [9]:
from api_key import *

generator_models = {
    "chatgpt": "gpt-3.5-turbo",
    "gpt-4": "gpt-4",
}

# API_KEY = ""
# ORGANIZATION = ""

openai.api_key = API_KEY

engine = "chatgpt"

In [10]:
PROMPT = "I am looking for occurrences of the <SKILL_TYPE> '<NAME>' in a document. However, the author doesn't always refer to this <SKILL_TYPE> using the full name. Generate only a list of 10 other names that I could look for, separated by commas."

EXAMPLE_DICT = {
    "technology": [
        "Microsoft Excel",
        "Excel, MS Excel, Microsoft Excel, Spreadsheet software by Microsoft, Microsoft's spreadsheet application, Excel program, Excel software, Microsoft's data analysis tool, Microsoft's workbook software, Spreadsheet program by Microsoft",
    ],
    "certification": [
        "AWS DevOps Engineer",
        "AWS, AWS DevOps Specialist, Amazon DevOps Engineer, AWS DevOps Practitioner, Certified AWS DevOps Professional, AWS DevOps Architect, Amazon Web Services DevOps Expert, AWS DevOps Solutions Engineer, AWS Cloud DevOps Engineer, AWS DevOps Deployment Specialist, AWS DevOps Integration Engineer",
    ],
}


class Generator:
    def __init__(self):
        self.model = generator_models[engine]

    def generate(self, skill_type, skill_name):
        messages = []
        sys_message = {
            "role": "system",
            "content": f"You are an expert at human resources, specialized in the IT domain.",
        }
        messages.append(sys_message)

        # Get the prompt

        question_example_content = PROMPT.replace("<SKILL_TYPE>", skill_type).replace(
            "<NAME>", EXAMPLE_DICT[skill_type][0]
        )
        example_answer = EXAMPLE_DICT[skill_type][1]
        question_content = PROMPT.replace("<SKILL_TYPE>", skill_type).replace(
            "<NAME>", skill_name
        )
        messages.append({"role": "user", "content": question_example_content})
        messages.append({"role": "assistant", "content": example_answer})
        messages.append({"role": "user", "content": question_content})
        flag = True
        while flag:  # while getting exception from API, retry
            try:
                response = openai.ChatCompletion.create(
                    model=self.model,
                    messages=messages,
                    top_p=1.0,
                    temperature=0.8,
                    frequency_penalty=0.8,
                    presence_penalty=0.5,
                )
                flag = False
            except Exception as e:
                print(f"The following error has happened. Waiting for 5seconds:\n{e}")
                time.sleep(5)
        output_text = response["choices"][0]["message"]["content"]
        return output_text

In [10]:
generator = Generator()

tech_certif_lang = pd.read_csv("taxonomy/tech_certif_lang.csv")
tech_certif_lang.drop(columns=["Level 1.5"], inplace=True)
tech = tech_certif_lang[tech_certif_lang['Level 1'] == "Technologies"]
tech['alternative_names'] = tech['Level 2'].apply(lambda name: generator.generate('technologie', name ))

certif = tech_certif_lang[tech_certif_lang["Level 1"] == "Certifications"]
certif["alternative_names"] = certif["Level 2"].apply(
    lambda name: generator.generate("certification", name)
)

/tmp/ipykernel_779043/232547023.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  certif['alternative_names'] = certif['Level 2'].apply(lambda name: generator.generate('certification', name ))


In [11]:
def clean_list(row):
    skill_name = row["Level 2"]
    skill_list = row["alternative_names"]
    skill_list = skill_list.replace("\n", ", ")
    skill_list = skill_list.split(":")[1] if ":" in skill_list else skill_list
    skill_list = re.sub(r"\d+\. ", "", skill_list)
    skill_list = skill_list.split(", ")
    skill_list = [
        skill for skill in skill_list if skill != "" and skill_name not in skill
    ]
    skill_list = ", ".join(skill_list)
    return skill_list

In [12]:
# tech['alternative_names_clean'] = tech.apply(clean_list, axis=1)
# to_save = tech[['Level 2', 'alternative_names_clean']]
# to_save.to_csv('taxonomy/technologies_alternative_names.csv', index=False, sep='\t')

certif["alternative_names_clean"] = certif.apply(clean_list, axis=1)
to_save = certif[["Level 2", "alternative_names_clean"]]
# to_save.to_csv('taxonomy/technologies_alternative_names.csv', index=False, sep='\t')

to_save.to_csv("taxonomy/certifications_alternative_names.csv", index=False, sep="\t")

/tmp/ipykernel_779043/1854031658.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  certif['alternative_names_clean'] = certif.apply(clean_list, axis=1)


In [37]:

pattern = r'\((.*?)\)'
def get_name(certif):
    matches = re.findall(pattern, certif)
    if matches:
        return matches[0]
    else:
        return ""

tech_certif_lang = pd.read_csv("../../data/taxonomy/tech_certif_lang.csv")
certif = tech_certif_lang[tech_certif_lang["Level 1"] == "Certifications"]

altnames = pd.read_csv("../../data/taxonomy/certifications_alternative_names.csv", sep="\t")
smaller_name = [get_name(name) for name in list(certif["Level 1.5"])]
smaller_name2 = [get_name(name) for name in list(certif["Level 2"])]

In [42]:
for index, row in altnames.iterrows():
    if smaller_name[index] != "":
        row['alternative_names_clean'] += ', ' + smaller_name[index]
    if smaller_name2[index] != "":
        row['alternative_names_clean'] += ', ' + smaller_name2[index]


In [43]:
altnames.to_csv("../../data/taxonomy/certifications_alternative_names.csv", index=False, sep="\t")

# merge taxonomy with ESCO



In [5]:
taxonomy = pd.read_csv("../data/taxonomy/taxonomy_V4.csv", sep=",")
taxonomy.columns
taxonomy.head()

,unique_id,ElementID,Dimension FE,Type Level 1,Type Level 1 E,Type Level 1 F,Type Level 2,Type Level 2 E,Type Level 2 F,Type Level 3,...,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43
0,53,2.1,Deskriptor,Kompetenzelemente,General Skills & Attitudes,Éléments de Compétence,Kognitive Fertigkeit,Cognitive Skill,Aptitude Cognitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,87,2.1.1,Deskriptor,Kompetenzelemente,General Skills & Attitudes,Éléments de Compétence,Kognitive Fertigkeit,Cognitive Skill,Aptitude Cognitive,Problemlösungsfähigkeit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,86,2.1.1.1,Deskriptor,Kompetenzelemente,General Skills & Attitudes,Éléments de Compétence,Kognitive Fertigkeit,Cognitive Skill,Aptitude Cognitive,Problemlösungsfähigkeit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120,2.1.1.2,Deskriptor,Kompetenzelemente,General Skills & Attitudes,Éléments de Compétence,Kognitive Fertigkeit,Cognitive Skill,Aptitude Cognitive,Problemlösungsfähigkeit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2.1.1.3,Deskriptor,Kompetenzelemente,General Skills & Attitudes,Éléments de Compétence,Kognitive Fertigkeit,Cognitive Skill,Aptitude Cognitive,Problemlösungsfähigkeit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
esco = pd.read_csv("../data/taxonomy/ESCO_v1.1.1 _de/skills_de.csv", sep=",")
print(len(esco)) #13896
esco = esco[esco["skillType"]=='knowledge']
print(len(esco)) #3059
# number of missing values per column
esco.isnull().sum()
esco = esco[['conceptUri', 'preferredLabel',  'altLabels', 'description']]

relationskill = pd.read_csv("../data/taxonomy/ESCO_v1.1.1 _de/broaderRelationsSkillPillar_de.csv", sep=",")
skillshierarchy = pd.read_csv("../data/taxonomy/ESCO_v1.1.1 _de/skillsHierarchy_de.csv", sep=",")

esco.head()

13896
3059


,conceptUri,preferredLabel,altLabels,description
7,http://data.europa.eu/esco/skill/000f1d3d-220f...,Haskell,NaN,Techniken und Grundsätze der Softwareentwicklu...
16,http://data.europa.eu/esco/skill/00506f28-e884...,Sport- und Bewegungsmedizin,NaN,Vorbeugung und Behandlung von Verletzungen ode...
24,http://data.europa.eu/esco/skill/00920055-eb50...,Unternehmensrisikomanagement,NaN,Eine auf einem Plan basierende Unternehmensstr...
30,http://data.europa.eu/esco/skill/00c04e40-35ea...,inkrementelle Entwicklung,NaN,Das Modell der inkrementellen Entwicklung ist ...
31,http://data.europa.eu/esco/skill/00c51318-4ea9...,Hilfsmittel für Alltagsarbeiten,NaN,"Arten von Spezialausrüstung, Prothesen und Ort..."


In [76]:
skillshierarchy.head()

,Level 0 URI,Level 0 preferred term,Level 1 URI,Level 1 preferred term,Level 2 URI,Level 2 preferred term,Level 3 URI,Level 3 preferred term,Description,Scope note,Level 0 code,Level 1 code,Level 2 code,Level 3 code
0,http://data.europa.eu/esco/skill/e35a5936-091d...,Sprachliche Fähigkeiten und Kenntnisse,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,L,NaN,NaN,NaN
1,http://data.europa.eu/esco/skill/e35a5936-091d...,Sprachliche Fähigkeiten und Kenntnisse,http://data.europa.eu/esco/skill/43f425aa-f45d...,Sprachen,NaN,NaN,,NaN,"Ability to communicate through reading, writin...",NaN,L,L1,NaN,NaN
2,http://data.europa.eu/esco/skill/e35a5936-091d...,Sprachliche Fähigkeiten und Kenntnisse,http://data.europa.eu/esco/skill/e434e71a-f068...,Alte Sprachen,NaN,NaN,,NaN,"All dead languages, no longer actively used, o...",Excludes:\n- all languages that are actively ...,L,L2,NaN,NaN
3,http://data.europa.eu/esco/skill/04a13491-b58c...,Querschnittsfähigkeiten und kompetenzen,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,T,NaN,NaN,NaN
4,http://data.europa.eu/esco/skill/04a13491-b58c...,Querschnittsfähigkeiten und kompetenzen,http://data.europa.eu/esco/skill/021a23e1-907e...,Fähigkeiten und Kompetenzen im Bereich Selbstm...,NaN,NaN,,NaN,Skills and competences requiring individuals t...,NaN,T,T3,NaN,NaN


In [63]:
uritoexclude = 'http://data.europa.eu/esco/isced-f/06'
relationskill1 = relationskill[relationskill['conceptUri']==uritoexclude]
print(len(relationskill1)) # 1: broaderUri = 'http://data.europa.eu/esco/skill/c46fcb45-5c14-4ffa-abed-5a43f104bb22'
relationskill2 = relationskill[relationskill['broaderUri']==uritoexclude]
print(len(relationskill2)) # 2: conceptUri = http://data.europa.eu/esco/isced-f/061, http://data.europa.eu/esco/isced-f/068
esco1 = esco[esco['conceptUri']==uritoexclude]
print(len(esco1)) # 0
skillshierarchy1 = skillshierarchy[skillshierarchy['Level 0 URI']==uritoexclude]
print(len(skillshierarchy1)) # 0
skillshierarchy2 = skillshierarchy[skillshierarchy['Level 1 URI']==uritoexclude]
print(len(skillshierarchy2)) # 9: Level 0 URI = 'http://data.europa.eu/esco/skill/c46fcb45-5c14-4ffa-abed-5a43f104bb22' and Level 2 URI = http://data.europa.eu/esco/isced-f/061, http://data.europa.eu/esco/isced-f/068
skillshierarchy3 = skillshierarchy[skillshierarchy['Level 2 URI']==uritoexclude]
print(len(skillshierarchy3)) # 0
skillshierarchy4 = skillshierarchy[skillshierarchy['Level 3 URI']==uritoexclude]
print(len(skillshierarchy4)) # 0

1
2
0
0
9
0
0


In [79]:
print(len(esco[esco['conceptUri'].isin(list(relationskill['broaderUri']))]))
print(len(esco[esco['conceptUri'].isin(list(relationskill['conceptUri']))]))
for level in ['Level 0 URI', 'Level 1 URI', 'Level 2 URI', 'Level 3 URI']:
    print(len(esco[esco['conceptUri'].isin(list(skillshierarchy[level]))]))
# esco['conceptUri'] looks like this: http://data.europa.eu/esco/skill/000f1d3d-220f...

# conclusion: esco['conceptUri'] are all in relationskill['conceptUri'] --> can deduce broaderURi

for level in ['Level 0 URI', 'Level 1 URI', 'Level 2 URI', 'Level 3 URI']:
    print(len(relationskill[relationskill['broaderUri'].isin(list(skillshierarchy[level]))]))

# conclusion: the broaderUri mostly corresponds to skillshierarchy['Level 3 URI']

# the uritoexclude = 'http://data.europa.eu/esco/isced-f/06' is in skillshierarchy['Level 1 URI']
# The associated skillshierarchy['Level 3 URI'] are:
broaderskill_toexclude = list(skillshierarchy[skillshierarchy['Level 1 URI']==uritoexclude]['Level 3 URI'])
print(broaderskill_toexclude)
skillshierarchy['Level 3 URI'] = skillshierarchy['Level 3 URI'].fillna('')
print(list(skillshierarchy[skillshierarchy['Level 3 URI'].str.startswith(uritoexclude)]['Level 3 URI']))
# these two lists are the same, so the pattern is right

# get conceptUri of broader skills to exclude:
conceptUri_toexclude = list(relationskill[relationskill['broaderUri'].isin(broaderskill_toexclude)]['conceptUri'])
print(len(conceptUri_toexclude))
len(esco[esco['conceptUri'].isin(conceptUri_toexclude)])
esco_filtered = esco[~esco['conceptUri'].isin(conceptUri_toexclude)]
print(len(esco_filtered)) # 2741



322
3059
0
0
0
0
28
229
633
13201
['', '', 'http://data.europa.eu/esco/isced-f/0610', 'http://data.europa.eu/esco/isced-f/0611', 'http://data.europa.eu/esco/isced-f/0612', 'http://data.europa.eu/esco/isced-f/0613', 'http://data.europa.eu/esco/isced-f/0619', '', 'http://data.europa.eu/esco/isced-f/0688']
['http://data.europa.eu/esco/isced-f/0610', 'http://data.europa.eu/esco/isced-f/0611', 'http://data.europa.eu/esco/isced-f/0612', 'http://data.europa.eu/esco/isced-f/0613', 'http://data.europa.eu/esco/isced-f/0619', 'http://data.europa.eu/esco/isced-f/0688']
318
2741


In [104]:
# merge skillshierarchy with esco
# 1/ get broader URi
esco_filtered_augmented = pd.merge(esco_filtered, relationskill, left_on='conceptUri', right_on='conceptUri', how='left')
print(len(esco_filtered_augmented)) # 3281
esco_filtered_augmented = esco_filtered_augmented[['conceptUri', 'preferredLabel', 'altLabels', 'description', 'broaderUri']]
# After merging we get duplicated conceptURi
# 2/ get skill levels
esco_filtered_augmented2 = pd.merge(esco_filtered_augmented, skillshierarchy, left_on='broaderUri', right_on='Level 3 URI', how='left')
# remove NAs
esco_filtered_augmented2.isnull().sum()
len(esco_filtered_augmented2['conceptUri'].unique())
esco_filtered_augmented2.dropna(subset=['Level 0 URI'], inplace=True)
len(esco_filtered_augmented2['conceptUri'].unique())
print(esco_filtered_augmented2.isnull().sum())

print(len(esco_filtered_augmented2)) 
esco_filtered_augmented2 = esco_filtered_augmented2[['conceptUri', 'preferredLabel', 'altLabels', 'description', 'Level 0 preferred term', 'Level 1 preferred term', 'Level 2 preferred term', 'Level 3 preferred term']]

3281
conceptUri                   0
preferredLabel               0
altLabels                 1812
description                  0
broaderUri                   0
Level 0 URI                  0
Level 0 preferred term       0
Level 1 URI                  0
Level 1 preferred term       0
Level 2 URI                  0
Level 2 preferred term       0
Level 3 URI                  0
Level 3 preferred term       0
Description                186
Scope note                2661
Level 0 code                 0
Level 1 code              2661
Level 2 code              2661
Level 3 code              2661
dtype: int64
2662


In [105]:
def concatenate_cols(row):
    output = row["preferredLabel"] if not pd.isna(row["preferredLabel"]) else ""
    output += f" {row['altLabels']}" if not pd.isna(row["altLabels"]) else ""
    #output += f" {row['hiddenLabels']}" if not pd.isna(row["hiddenLabels"]) else ""
    return output

#esco_filtered_augmented2["preferredLabel"] = esco_filtered_augmented2.apply(lambda row: concatenate_cols(row), axis=1)

esco_filtered_augmented2.rename(columns={'preferredLabel': 'Type Level 4', 'description': 'Definition', 'Level 1 preferred term': 'Type Level 1', 'Level 2 preferred term': 'Type Level 2', 'Level 3 preferred term': 'Type Level 3', 'conceptUri': 'Source', 'Level 0 preferred term': 'Dimension'}, inplace=True)

esco_filtered_augmented2["unique_id"] = [1000+ i for i in range(len(esco_filtered_augmented2))]



esco_filtered_augmented2.head()

,Source,Type Level 4,altLabels,Definition,Dimension,Type Level 1,Type Level 2,Type Level 3,unique_id
0,http://data.europa.eu/esco/skill/00506f28-e884...,Sport- und Bewegungsmedizin,NaN,Vorbeugung und Behandlung von Verletzungen ode...,Kenntnisse,Gesundheit und soziale Dienste,Gesundheit,Medizin,1000
2,http://data.europa.eu/esco/skill/00920055-eb50...,Unternehmensrisikomanagement,NaN,Eine auf einem Plan basierende Unternehmensstr...,Kenntnisse,"Wirtschaft, Verwaltung und Rechtswissenschaft",Wirtschaft und Verwaltung,Management und Verwaltung,1001
4,http://data.europa.eu/esco/skill/00c51318-4ea9...,Hilfsmittel für Alltagsarbeiten,NaN,"Arten von Spezialausrüstung, Prothesen und Ort...",Kenntnisse,Gesundheit und soziale Dienste,Gesundheit,Therapie und Rehabilitation,1002
5,http://data.europa.eu/esco/skill/00c85cbc-2b24...,Sägetechniken,NaN,Verschiedene Sägetechniken zur Verwendung manu...,Kenntnisse,"Ingenieurwesen, Fertigung und Bauwesen",Fertigung und Verarbeitung,"Werkstoffe (Glas, Papier, Plastik und Holz)",1003
6,http://data.europa.eu/esco/skill/00eefa32-9f77...,Kaltvulkanisation,Kaltvulkanisierung,"Technik zur Reparatur defekter Reifen, insbeso...",Kenntnisse,"Ingenieurwesen, Fertigung und Bauwesen",Ingenieurwesen und Ingenieurberufe,Chemietechnik und ‑verfahren,1004


In [106]:
# append dataframe to taxonomy
taxonomy2 = taxonomy.append(esco_filtered_augmented2, ignore_index=True)
print(len(taxonomy2))
taxonomy2.tail()


3035


/tmp/ipykernel_96683/4141180993.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  taxonomy2 = taxonomy.append(esco_filtered_augmented2, ignore_index=True)


,unique_id,ElementID,Dimension FE,Type Level 1,Type Level 1 E,Type Level 1 F,Type Level 2,Type Level 2 E,Type Level 2 F,Type Level 3,...,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,altLabels,Dimension
3030,3657,NaN,NaN,"Ingenieurwesen, Fertigung und Bauwesen",NaN,NaN,Ingenieurwesen und Ingenieurberufe,NaN,NaN,Mechanik und Metallbearbeitung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bohrkronentypen,Kenntnisse
3031,3658,NaN,NaN,Dienstleistungen,NaN,NaN,Dienstleistungen im Bereich Verkehr,NaN,NaN,Dienstleistungen im Bereich Verkehr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kenntnisse
3032,3659,NaN,NaN,Kunst und Geisteswissenschaften,NaN,NaN,Kunst,NaN,NaN,Kunsthandwerk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kenntnisse
3033,3660,NaN,NaN,Gesundheit und soziale Dienste,NaN,NaN,Soziale Dienste,NaN,NaN,Sozialarbeit und Beratung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abhängigkeit von psychotropen Substanzen\nDrog...,Kenntnisse
3034,3661,NaN,NaN,Kunst und Geisteswissenschaften,NaN,NaN,Fremdsprachen,NaN,NaN,Fremdsprachenerwerb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verarbeitung natürlicher Sprachen\nNatural Lan...,Kenntnisse


In [108]:
taxonomy2.to_csv("taxonomy_with_esco.csv", index=False, sep=",")

# load jobs and course data

- https://evrlearn-staging.herokuapp.com/api/v1/proto/learning_opportunities - list of courses
- https://evrlearn-staging.herokuapp.com/api/v1/proto/vacancies - list of parsed jobs
- https://evrlearn-staging.herokuapp.com/api/v1/proto/job_skills - list of available job skills
- https://evrlearn-staging.herokuapp.com/api/v1/proto/job_profiles - list of job profiles with related skills 


### Run from the protosp01 directory:

```bash
bash update_platform_data.sh
```

In [1]:
import os, re
import pyconll
import json
import pandas as pd
import random
import openai
import os
import pandas as pd
import re
import json
from tqdm import tqdm


In [21]:
# !wget https://evrlearn-staging.herokuapp.com/api/v1/proto/learning_opportunities -O "/home/montario/Documents/EPFL - NLP/SCESC/code/raw/learning_opportunities"
# !wget https://evrlearn-staging.herokuapp.com/api/v1/proto/vacancies -O "/home/montario/Documents/EPFL - NLP/SCESC/code/raw/vacancies"

In [22]:
# load json file
job_profiles = pd.read_json("raw/job_profiles.json")
job_profiles.head()
len(job_profiles)

460

In [23]:
job_skills = pd.read_json("raw/job_skills.json")
len(job_skills)  # 35

398

In [13]:
courses = pd.read_json("../data/raw/learning_opportunities.json")
print(len(courses))  # 5738
# TODO take only active = True courses
print(len(courses[courses["active"] == True]))  # 4549
courses.columns

# TODO missing data
print(courses.isna().sum())
courses.head()

for col in courses.columns:
    print(col, courses[col][0])

5738
4549
id                                        0
name                                      0
type                                      0
intro                                    27
active                                    0
pricing_description                    1630
ects_points                            4326
average_effort_per_week                4559
total_effort                            498
key_benefits                             17
learning_targets_description             10
target_group_description               1619
structure_description                  1621
application_process_description        1705
admission_criteria_description         1665
required_number_years_of_experience    5153
level                                  3590
certificate_type                        490
currency                                 79
pricing_type                             10
transaction_type                       3556
dtype: int64
id 744
name CAS Haftpflicht- und Versicherungsrecht
t

In [25]:
jobs = pd.read_json("raw/vacancies.json", encoding="utf-8")
print(len(jobs))  # 3759 --> 6616
jobs.head()

7631


,id,name,orig_url,company_name,company_url,lead,description
0,4623,Business Analyst Leistungen,https://www.jobs.ch/en/vacancies/detail/70cabf...,ELCA Informatik AG,https://www.jobs.ch/en/companies/11529-elca-in...,None,Um unser Wachstum weiterhin erfolgreich fortzu...
1,15,Securities Master Data Specialist (Ref. 2177),https://www.jobs.ch/en/vacancies/detail/0f8346...,None,None,None,"<!DOCTYPE html>\n<html lang=""en"" data-template..."
2,2226,IT Supporter (m/w),https://www.jobs.ch/en/vacancies/detail/e72b68...,univativ Schweiz AG,https://www.jobs.ch/en/companies/95851-univati...,"Ob Student, Berufseinsteiger oder Berufserfahr...",Karrierelevel: Berufseinsteiger Pensum: 80-10...
3,5,Scrum Master/-in & Applikations-Analyst/-in Ep...,https://www.jobs.ch/en/vacancies/detail/9a1fa7...,None,None,None,Das Luzerner Kantonsspital mit den Standorten ...
4,16,"Product Owner ""Financial Messaging"" for Multib...",https://www.jobs.ch/en/vacancies/detail/703709...,None,None,None,"<!DOCTYPE html>\n<html lang=""en"" data-template..."


In [26]:
jobs["fulltext"] = jobs["description"] + jobs["name"]
courses["fulltext"] = (
    courses["structure_description"].fillna("")
    + courses["name"]
    + courses["key_benefits"].fillna("")
    + courses["intro"].fillna("")
)


def count_sentences(text):
    sentences = re.split(r"(?<=[.!?]) +", text)
    return len(sentences)


price_per_sentence_gpt3 = 0.035 / 32
price_per_sentence_gpt4 = 1.2 / 32

# Apply the function to create a new column 'num_sentences'
for dfname, df in zip(["jobs", "courses"], [jobs, courses]):
    df["num_sentences"] = df["fulltext"].apply(count_sentences)

    # Calculate the average number of sentences
    average_sentences = df["num_sentences"].mean()
    total_sentences = df["num_sentences"].sum()

    price_with_gpt3 = total_sentences * price_per_sentence_gpt3
    prince_with_gpt4 = total_sentences * price_per_sentence_gpt4

    print(dfname, ": \n")
    print(f"Average number of sentences: {average_sentences:.2f}")
    print(f"Total number of sentences: {total_sentences:.2f}")
    print(f"price_with_gpt3: {price_with_gpt3:.2f}")
    print(f"prince_with_gpt4: {prince_with_gpt4:.2f}")
    print("\n")

jobs : 

Average number of sentences: 12.31
Total number of sentences: 93935.00
price_with_gpt3: 102.74
prince_with_gpt4: 3522.56


courses : 

Average number of sentences: 6.35
Total number of sentences: 36412.00
price_with_gpt3: 39.83
prince_with_gpt4: 1365.45




In [27]:
cols = ["structure_description", "name", "key_benefits", "intro"]
for col in cols:
    print(col, courses[col].isna().sum())

cols = ["description", "name"]
for col in cols:
    print(col, jobs[col].isna().sum())

structure_description 1621
name 0
key_benefits 17
intro 27
description 0
name 0


In [28]:
courses.describe()

,id,ects_points,average_effort_per_week,total_effort,required_number_years_of_experience,num_sentences
count,5738.000000,1412.000000,1179.000000,5240.000000,585.000000,5738.000000
mean,5220.989718,25.844901,12.579304,154.477672,2.842735,6.345765
std,2643.716068,237.474303,22.024206,355.025285,1.698877,7.225278
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3788.250000,10.000000,5.000000,3.000000,2.000000,3.000000
50%,5781.500000,15.000000,10.000000,16.000000,2.000000,5.000000
75%,7471.750000,15.000000,16.000000,130.000000,3.000000,8.000000
max,8906.000000,8900.000000,450.000000,5400.000000,15.000000,205.000000


In [30]:
# sample 20 sentences:
jobs_sampled = jobs.sample(30)["description"].tolist()
jobs_sampled = [text for text in jobs_sampled if detect_language(text) == "de"]
jobs_sampled = [re.split(r"(?<=[.!?]) +", job) for job in jobs_sampled]
jobs_sampled = [
    random.sample([sent for sent in sent_list if 20 < len(sent) < 10000], 1)[0]
    for sent_list in jobs_sampled
]
print(jobs_sampled)
# jobs_json = [{'sentence': sent, 'extracted_skills': None} for sent in jobs_sampled]
# save as json
# with open("raw/job_sentences.json", "w", encoding="utf-8") as f:
#     json.dump(jobs_json, f, ensure_ascii=False, indent=4)

['Wir begleiten unsere Kunden von der ersten Idee bis hin zur Serienfertigung inmittleren bis grossen Stückzahlen und suchen per sofort Verstärkung für unser Team als: Leiter*in Qualitätsmanagement / Entwicklungsingenieur*in  Deine Hauptaufgaben:   \n Durchführen der QM-Maßnahmen gemäss DIN ISO 9001:2015 \n Koordinieren von Aktivitäten zur Überwachung und Verbesserung des QM-Systems sowie Kontrolleder Umsetzung der beschlossenen Massnahmen \n Bearbeiten von Reklamationen und Fehlern (intern und extern).', 'Mehr als 650 Mitarbeitende an unserem Hauptsitz in Zürich sowie in den Büros in Bern, Lausanne, Budapest, Lissabon, Singapur und Ho-Chi-Minh-Stadt entwickeln Lösungen, die unser Versprechen «And your digital business works» an unsere Kunden erfüllen.', 'Auch im Vertrieb von ganzheitlichen Lösungen und im IT-Projektgeschäft sind wir ein bekannter Ansprechpartner.', 'Klicke dafür auf «Jetzt bewerben».', 'Corporate Social Responsibility sowie DEI-Standards (Diversity, Ethics and Inclusi

# analyze job role

In [4]:
jobs = pd.read_json("../data/raw/vacancies.json", encoding="utf-8")
print(len(jobs))  # 3759 --> 6616
jobs.head()

15901


,id,name,orig_url,company_name,company_url,lead,description
0,4623,Business Analyst Leistungen,https://www.jobs.ch/en/vacancies/detail/70cabf...,ELCA Informatik AG,https://www.jobs.ch/en/companies/11529-elca-in...,None,Um unser Wachstum weiterhin erfolgreich fortzu...
1,15,Securities Master Data Specialist (Ref. 2177),https://www.jobs.ch/en/vacancies/detail/0f8346...,None,None,None,"<!DOCTYPE html>\n<html lang=""en"" data-template..."
2,2226,IT Supporter (m/w),https://www.jobs.ch/en/vacancies/detail/e72b68...,univativ Schweiz AG,https://www.jobs.ch/en/companies/95851-univati...,"Ob Student, Berufseinsteiger oder Berufserfahr...",Karrierelevel: Berufseinsteiger Pensum: 80-10...
3,5,Scrum Master/-in & Applikations-Analyst/-in Ep...,https://www.jobs.ch/en/vacancies/detail/9a1fa7...,None,None,None,Das Luzerner Kantonsspital mit den Standorten ...
4,16,"Product Owner ""Financial Messaging"" for Multib...",https://www.jobs.ch/en/vacancies/detail/703709...,None,None,None,"<!DOCTYPE html>\n<html lang=""en"" data-template..."


In [33]:
def clean_job_role(role):
    role = role.replace('IT', '').replace('ICT', '')
    role = role.lower()
    role = role.replace('ref', '')
    role = role.replace("m/w/d", "").replace("m/f/d", "").replace("m/f/x", "").replace('m/w', '').replace('w/m/d','').replace('w/m', '')
    # remove numbers
    role = re.sub(r"\d+", "", role)
    # remove punctuation
    role = re.sub(r"[^\w\s]", "", role)
    # remove feminine
    role = re.sub(r'\b(\w+)in\b', r'\1', role)
    # remove single letters
    role = re.sub(r'\b\w\b', '', role)
    # remove extra spaces
    role = re.sub(r'\s+', ' ', role).strip()

    # translations
    role = role.replace('applikationsmanager', 'application manager')
    role = role.replace('applikationsentwickler', 'application developer')
    role = role.replace('projektleiter', 'project manager')
    role = role.replace('systemadministrator', 'system administrator')
    # role = role.replace('informatiker', '') ??
    # role = role.replace('techniker', '') ??
    role = role.replace('systemtechniker', 'system engineer')
    role = role.replace('softwareentwickler', 'software engineer')
    role = role.replace('software entwickler', 'software engineer')
    role = role.replace('service desk mitarbeiter', 'helpdesk')
    role = role.replace('systemspezialist', 'system specialist')
    role = role.replace('helpdesk mitarbeiter', 'helpdesk')
    role = role.replace('mitarbeiter support', 'supporter')
    role = role.replace('support', 'supporter')
    role = role.replace('supporterer', 'supporter')
    role = role.replace('supporter engineer', 'supporter')
    return role

jobs["job_role"] = jobs["name"].apply(clean_job_role)
jobs["job_role"].value_counts().head(50)

job_role
system engineer                         440
supporterer                             393
system administrator                    142
application manager                     134
software engineer                        99
project manager                          83
senior system engineer                   80
business analyst                         70
network engineer                         59
junior system engineer                   58
senior software engineer                 51
helpdesk                                 49
praktikant junior informatiker           48
security engineer                        45
junior supporterer                       44
data engineer                            38
scrum master                             37
supporter engineer                       32
java software engineer                   30
solution architect                       29
devops engineer                          29
informatiker                             28
senior network engineer

In [44]:
# Rollouter job

jobs[jobs["name"].str.contains("support")]
jobs[jobs["job_role"].str.contains("rollouter")]
list_rollouter_job = jobs[jobs["name"].str.contains("ollout")]["id"].to_list()
with open("../protosp01/skillExtract/list_rollouter_job_ids.json", "w", encoding="utf-8") as f:
    json.dump(list_rollouter_job, f)

with open("../protosp01/skillExtract/list_rollouter_job_ids.txt", "w", encoding="utf-8") as f:
    for item in list_rollouter_job:
        f.write("https://evrlearn-staging.herokuapp.com/admin/vacancies/%s\n" % item)

In [56]:
# solution architect job:
jobs[jobs["job_role"].str.contains("solution architect")]
list_solution_architect_job = jobs[jobs["job_role"].str.contains("solution architect")]["id"].to_list()
with open("../protosp01/skillExtract/list_solution_architect_job_ids.json", "w", encoding="utf-8") as f:
    json.dump(list_solution_architect_job, f)

with open("../protosp01/skillExtract/list_solution_architect_job_ids.txt", "w", encoding="utf-8") as f:
    for item in list_solution_architect_job:
        f.write("https://evrlearn-staging.herokuapp.com/admin/vacancies/%s\n" % item)

